In [157]:
import string
import pandas as pd
import numpy as np
import re
import matplotlib.pyplot as plt
import nltk
from nltk.corpus import  stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from difflib import SequenceMatcher
import warnings
warnings.filterwarnings('ignore')

In [158]:
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('words')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\WB\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\WB\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package words to
[nltk_data]     C:\Users\WB\AppData\Roaming\nltk_data...
[nltk_data]   Package words is already up-to-date!


True

In [12]:
# from google.colab import drive
# drive.mount('/content/drive')

# Import the necessary modules
# !pip install language_tool_python
import language_tool_python


tool = language_tool_python.LanguageTool('en-US')

sentence = "The system shall  specific player id team."

matches = tool.check(sentence)

corrected_sentence = language_tool_python.utils.correct(sentence, matches)

# Print the corrected sentence
print(corrected_sentence)

The system shall specific player I'd team.


**preprocessing**

In [160]:
data = pd.read_csv('./Ubiquitous-Domain AI.csv', encoding='unicode-escape')
data

,Projects,Raw Requirements,Requiremnet Name,Type,Requirement with Standard Syntax(previous),Unnamed: 5,type,Requirement with Standard Syntax,Domain,sub_domain,Unnamed: 10
0,PR-01,The Game must be developed using the Unity Gam...,Game Engine,Ubiquitous,The game shall be developed using unity game e...,NaN,ubiquitous,The game shall be developed using unity game e...,Game,Mobile app,NaN
1,PR-01,The Game must be programmed using the C# langu...,Language,Ubiquitous,The game shall be written in C# and use Unity ...,NaN,ubiquitous,The game shall be written in C# and use Unity ...,Game,Mobile app,NaN
2,PR-01,The Game must feature accurate sound effects a...,Audio,Ubiquitous,The Game shall feature accurate sound effects ...,NaN,ubiquitous,The Game shall feature accurate sound effects ...,Game,Mobile app,NaN
3,PR-01,The Game must be playable using the keyboard k...,Control,Ubiquitous,The game shall be playable using mouse and key...,NaN,ubiquitous,The game shall be playable using mouse and key...,Game,Mobile app,NaN
4,PR-01,The Game must offer a compelling story which w...,Story,Ubiquitous,The Game shall offer a compelling story which ...,NaN,ubiquitous,The Game shall offer a compelling story which ...,Game,Mobile app,NaN
...,...,...,...,...,...,...,...,...,...,...,...
4628,PR-314,The system shall display the time spent by pla...,Produce Game Results,Ubiquitous,The system shall display the time spent by pla...,NaN,ubiquitous,The system shall display the time spent by pla...,NaN,NaN,NaN
4629,PR-314,The system shall be able to calculate and disp...,Produce Game Results,Ubiquitous,The system shall be able to calculate and disp...,NaN,ubiquitous,The system shall be able to calculate and disp...,NaN,NaN,NaN
4630,PR-315,The client will be able to scan the image from...,Scan,Ubiquitous,The system shall enable client to scan the ima...,NaN,ubiquitous,The system shall enable client to scan the ima...,NaN,NaN,NaN
4631,PR-315,The client will be able to rotate the 3D objec...,Rotation,Ubiquitous,The system shall enable client to rotate the 3...,NaN,ubiquitous,The system shall enable client to rotate the 3...,NaN,NaN,NaN


In [161]:
data.columns
# printing all the columns headers

Index(['Projects', 'Raw Requirements', 'Requiremnet Name', 'Type',
       'Requirement with Standard Syntax(previous)', 'Unnamed: 5', ' type',
       'Requirement with Standard Syntax', 'Domain', 'sub_domain',
       'Unnamed: 10'],
      dtype='object')

In [162]:
data = data[['Raw Requirements', 'Requirement with Standard Syntax', ' type', 'Domain', 'sub_domain']]
# selecting only the required columns

In [163]:
data
# printing the data

,Raw Requirements,Requirement with Standard Syntax,type,Domain,sub_domain
0,The Game must be developed using the Unity Gam...,The game shall be developed using unity game e...,ubiquitous,Game,Mobile app
1,The Game must be programmed using the C# langu...,The game shall be written in C# and use Unity ...,ubiquitous,Game,Mobile app
2,The Game must feature accurate sound effects a...,The Game shall feature accurate sound effects ...,ubiquitous,Game,Mobile app
3,The Game must be playable using the keyboard k...,The game shall be playable using mouse and key...,ubiquitous,Game,Mobile app
4,The Game must offer a compelling story which w...,The Game shall offer a compelling story which ...,ubiquitous,Game,Mobile app
...,...,...,...,...,...
4628,The system shall display the time spent by pla...,The system shall display the time spent by pla...,ubiquitous,NaN,NaN
4629,The system shall be able to calculate and disp...,The system shall be able to calculate and disp...,ubiquitous,NaN,NaN
4630,The client will be able to scan the image from...,The system shall enable client to scan the ima...,ubiquitous,NaN,NaN
4631,The client will be able to rotate the 3D objec...,The system shall enable client to rotate the 3...,ubiquitous,NaN,NaN


In [164]:
data.dropna(inplace=True)
# dropping the rows with null values in any column
data

,Raw Requirements,Requirement with Standard Syntax,type,Domain,sub_domain
0,The Game must be developed using the Unity Gam...,The game shall be developed using unity game e...,ubiquitous,Game,Mobile app
1,The Game must be programmed using the C# langu...,The game shall be written in C# and use Unity ...,ubiquitous,Game,Mobile app
2,The Game must feature accurate sound effects a...,The Game shall feature accurate sound effects ...,ubiquitous,Game,Mobile app
3,The Game must be playable using the keyboard k...,The game shall be playable using mouse and key...,ubiquitous,Game,Mobile app
4,The Game must offer a compelling story which w...,The Game shall offer a compelling story which ...,ubiquitous,Game,Mobile app
...,...,...,...,...,...
3224,User is able to search the social hub accounts...,The system shall enable a user to search the s...,ubiquitous,Social media,Mobile app
3225,User can further see their specific social app...,The system shall enable a user to further see ...,ubiquitous,Social media,Mobile app
3226,User is able to start conversation with people...,The system shall enable a user to start conver...,ubiquitous,Social media,Mobile app
3227,User is able to send/receive messages.,The system shall enable a user to send/receive...,ubiquitous,Social media,Mobile app


In [165]:
data.info()
# printing the information about the data frame like the number of rows and columns, the number of non-null values in each column and the data type of each column

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1962 entries, 0 to 3228
Data columns (total 5 columns):
 #   Column                            Non-Null Count  Dtype 
---  ------                            --------------  ----- 
 0   Raw Requirements                  1962 non-null   object
 1   Requirement with Standard Syntax  1962 non-null   object
 2    type                             1962 non-null   object
 3   Domain                            1962 non-null   object
 4   sub_domain                        1962 non-null   object
dtypes: object(5)
memory usage: 92.0+ KB


In [166]:
data.isnull().sum()
# printing the number of null values in each column

Raw Requirements                    0
Requirement with Standard Syntax    0
 type                               0
Domain                              0
sub_domain                          0
dtype: int64

In [167]:
data['raw_length'] = data['Raw Requirements'].str.len()
# creating a new column raw_length which contains the length of the raw requirements
data['standard_length'] = data['Requirement with Standard Syntax'].str.len()
# creating a new column standard_length which contains the length of the Requirement with Standard Syntax
data

,Raw Requirements,Requirement with Standard Syntax,type,Domain,sub_domain,raw_length,standard_length
0,The Game must be developed using the Unity Gam...,The game shall be developed using unity game e...,ubiquitous,Game,Mobile app,54,51
1,The Game must be programmed using the C# langu...,The game shall be written in C# and use Unity ...,ubiquitous,Game,Mobile app,100,74
2,The Game must feature accurate sound effects a...,The Game shall feature accurate sound effects ...,ubiquitous,Game,Mobile app,92,93
3,The Game must be playable using the keyboard k...,The game shall be playable using mouse and key...,ubiquitous,Game,Mobile app,59,52
4,The Game must offer a compelling story which w...,The Game shall offer a compelling story which ...,ubiquitous,Game,Mobile app,89,90
...,...,...,...,...,...,...,...
3224,User is able to search the social hub accounts...,The system shall enable a user to search the s...,ubiquitous,Social media,Mobile app,96,114
3225,User can further see their specific social app...,The system shall enable a user to further see ...,ubiquitous,Social media,Mobile app,63,88
3226,User is able to start conversation with people...,The system shall enable a user to start conver...,ubiquitous,Social media,Mobile app,74,92
3227,User is able to send/receive messages.,The system shall enable a user to send/receive...,ubiquitous,Social media,Mobile app,38,56


In [168]:
import nltk
nltk.download('omw-1.4')

[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\WB\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

In [169]:
data['Raw Requirements'] = data['Raw Requirements'].str.lower()
data['Raw Requirements'] = data['Raw Requirements'].str.replace(r'^.+@[^\.].*\.[a-z]{2,}$', 'emailaddress')
data['Raw Requirements'] = data['Raw Requirements'].str.replace(r'^http\://[a-zA-Z0-9\-\.]+\.[a-zA-Z]{2,3}(/\S*)?$', 'webaddress')
data['Raw Requirements'] = data['Raw Requirements'].str.replace(r'£|\$', 'dollers')
data['Raw Requirements'] = data['Raw Requirements'].str.replace(r'^\(?[\d]{3}\)?[\s-]?[\d]{3}[\s-]?[\d]{4}$', 'phonenumber')
data['Raw Requirements'] = data['Raw Requirements'].str.replace(r'\d+(\.\d+)?', 'numbr')
data['Raw Requirements'] = data['Raw Requirements'].apply(lambda x: ' '.join(term for term in x.split() if term not in string.punctuation))
stop_words = set(stopwords.words('english') + ['u', 'ü', 'ur', '4', '2', 'im', 'dont', 'doin', 'ure'])
data['Raw Requirements'] = data['Raw Requirements'].apply(lambda x: ' '.join(term for term in x.split() if term not in stop_words))
lemmatizer = WordNetLemmatizer()
data['Raw Requirements'] = data['Raw Requirements'].apply(lambda x: ' '.join(lemmatizer.lemmatize(t) for t in x.split()))

# first we convert all the words to lower case
# then we replace the email addresses, urls, phone numbers, currency symbols and numbers with their respective words to remove special characters
# stopwords are the words which do not add any meaning to a sentence like the, is, at, which, and etc and we are removing them
# then we remove the punctuation and the stop words and then we lemmatize the words


In [170]:
data['Requirement with Standard Syntax'] = data['Requirement with Standard Syntax'].str.lower()
data['Requirement with Standard Syntax'] = data['Requirement with Standard Syntax'].str.replace(r'^.+@[^\.].*\.[a-z]{2,}$', 'emailaddress')
data['Requirement with Standard Syntax'] = data['Requirement with Standard Syntax'].str.replace(r'^http\://[a-zA-Z0-9\-\.]+\.[a-zA-Z]{2,3}(/\S*)?$', 'webaddress')
data['Requirement with Standard Syntax'] = data['Requirement with Standard Syntax'].str.replace(r'£|\$', 'dollers')
data['Requirement with Standard Syntax'] = data['Requirement with Standard Syntax'].str.replace(r'^\(?[\d]{3}\)?[\s-]?[\d]{3}[\s-]?[\d]{4}$', 'phonenumber')
data['Requirement with Standard Syntax'] = data['Requirement with Standard Syntax'].str.replace(r'\d+(\.\d+)?', 'numbr')
data['Requirement with Standard Syntax'] = data['Requirement with Standard Syntax'].apply(lambda x: ' '.join(term for term in x.split() if term not in string.punctuation))
stop_words = set(stopwords.words('english') + ['u', 'ü', 'ur', '4', '2', 'im', 'dont', 'doin', 'ure'])
data['Requirement with Standard Syntax'] = data['Requirement with Standard Syntax'].apply(lambda x: ' '.join(term for term in x.split() if term not in stop_words))
lemmatizer = WordNetLemmatizer()
data['Requirement with Standard Syntax'] = data['Requirement with Standard Syntax'].apply(lambda x: ' '.join(lemmatizer.lemmatize(t) for t in x.split()))

In [171]:

data['clean_raw_length'] = data['Raw Requirements'].str.len()
data['clean_standard_length'] = data['Requirement with Standard Syntax'].str.len()
data

,Raw Requirements,Requirement with Standard Syntax,type,Domain,sub_domain,raw_length,standard_length,clean_raw_length,clean_standard_length
0,game must developed using unity game engine,game shall developed using unity game engine,ubiquitous,Game,Mobile app,54,51,43,44
1,game must programmed using c# language library...,game shall written c# use unity engine-compati...,ubiquitous,Game,Mobile app,100,74,70,60
2,game must feature accurate sound effect music ...,game shall feature accurate sound effect music...,ubiquitous,Game,Mobile app,92,93,74,75
3,game must playable using keyboard key mouse,game shall playable using mouse keyboard.,ubiquitous,Game,Mobile app,59,52,43,41
4,game must offer compelling story convey specia...,game shall offer compelling story convey speci...,ubiquitous,Game,Mobile app,89,90,62,63
...,...,...,...,...,...,...,...,...,...
3224,user able search social hub account people typ...,system shall enable user search social hub acc...,ubiquitous,Social media,Mobile app,96,114,66,81
3225,user see specific social app? public profile.,system shall enable user see specific social a...,ubiquitous,Social media,Mobile app,63,88,45,65
3226,user able start conversation people searched s...,system shall enable user start conversation pe...,ubiquitous,Social media,Mobile app,74,92,56,71
3227,user able send/receive messages.,system shall enable user send/receive messages.,ubiquitous,Social media,Mobile app,38,56,32,47


In [172]:

print ('Origian Raw Requirements Length', data['raw_length'].sum())
print ('Clean Raw Requirements Length', data['clean_raw_length'].sum())
print ('Origian Requirement with Standard Syntax', data['standard_length'].sum())
print ('Clean Requirement with Standard Syntax', data['clean_standard_length'].sum())

Origian Raw Requirements Length 164620
Clean Raw Requirements Length 121901
Origian Requirement with Standard Syntax 167631
Clean Requirement with Standard Syntax 126473


In [173]:
data['type'].value_counts()
# printing the number of requirements in each type (column name is type)

ubiquitous    1959
Ubiquitous       3
Name:  type, dtype: int64

In [174]:
data.reset_index(inplace=True)

In [175]:
for i in range(len(data[' type'])):
    str = data[' type'][i]
    if 'state' in str:
        data[' type'][i] = 'state-driven'
    elif 'eve' in str:
        data[' type'][i] = 'event-driven'
    elif 'biquit' in str:
        data[' type'][i] = 'ubiquitous'
    elif 'wanted' in str:
        data[' type'][i] = 'unwanted-behaviour'
    elif 'tional' in str:
        data[' type'][i] = 'optional-feature'

In [176]:

data[' type'].value_counts()

ubiquitous    1962
Name:  type, dtype: int64

In [177]:
data[' type'] = data[' type'].map({'ubiquitous':0, 'state-driven':1, 'event-driven':2, 'unwanted-behaviour':3, 'optional-feature':4})
# assigning the numbers to the types and storing them in the same column
# for example ubiquitous is assigned 0, state-driven is assigned 1 and so on
data

,index,Raw Requirements,Requirement with Standard Syntax,type,Domain,sub_domain,raw_length,standard_length,clean_raw_length,clean_standard_length
0,0,game must developed using unity game engine,game shall developed using unity game engine,0,Game,Mobile app,54,51,43,44
1,1,game must programmed using c# language library...,game shall written c# use unity engine-compati...,0,Game,Mobile app,100,74,70,60
2,2,game must feature accurate sound effect music ...,game shall feature accurate sound effect music...,0,Game,Mobile app,92,93,74,75
3,3,game must playable using keyboard key mouse,game shall playable using mouse keyboard.,0,Game,Mobile app,59,52,43,41
4,4,game must offer compelling story convey specia...,game shall offer compelling story convey speci...,0,Game,Mobile app,89,90,62,63
...,...,...,...,...,...,...,...,...,...,...
1957,3224,user able search social hub account people typ...,system shall enable user search social hub acc...,0,Social media,Mobile app,96,114,66,81
1958,3225,user see specific social app? public profile.,system shall enable user see specific social a...,0,Social media,Mobile app,63,88,45,65
1959,3226,user able start conversation people searched s...,system shall enable user start conversation pe...,0,Social media,Mobile app,74,92,56,71
1960,3227,user able send/receive messages.,system shall enable user send/receive messages.,0,Social media,Mobile app,38,56,32,47


In [178]:
import nltk
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
# punkt is a pre-trained tokenizer for english language and averaged_perceptron_tagger is a pre-trained part-of-speech tagger for english language

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\WB\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\WB\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [260]:
from collections import Counter
from nltk import word_tokenize, pos_tag, pos_tag_sents
tagged_texts = pos_tag_sents(map(word_tokenize, texts))
# pos_tag_sents is a function which takes a list of sentences and returns a list of tuples where each tuple contains a word and its part-of-speech tag

# for example from sentence "game must developed using unity game engine" we get the following list of tuples
# [('game', 'NN'), ('must', 'MD'), ('developed', 'VBN'), ('using', 'VBG'), ('unity', 'NN'), ('game', 'NN'), ('engine', 'NN')]
# where NN is noun, 
# MD is modal verb (modal verbs are can, could, may, might, must, will, would, shall, should, and needn't),
# VBN is past participle 
# VBG is gerund
tagged_texts

data['POS'] = tagged_texts
data

0       [(game, NN), (must, MD), (developed, VB), (usi...
1       [(game, NN), (must, MD), (programmed, VB), (us...
2       [(game, NN), (must, MD), (feature, VB), (accur...
3       [(game, NN), (must, MD), (playable, VB), (usin...
4       [(game, NN), (must, MD), (offer, VB), (compell...
                              ...                        
1957    [(user, RBR), (able, JJ), (search, NN), (socia...
1958    [(user, NN), (see, VBP), (specific, JJ), (soci...
1959    [(user, RBR), (able, JJ), (start, JJ), (conver...
1960    [(user, RBR), (able, JJ), (send/receive, JJ), ...
1961    [(user, RBR), (able, JJ), (delete, JJ), (messa...
Name: POS, Length: 1962, dtype: object

In [180]:
dataa = data[['index', 'POS']].copy()
# creating a new dataframe with only the index and POS columns

In [181]:

dataa

,index,POS
0,0,"[(game, NN), (must, MD), (developed, VB), (usi..."
1,1,"[(game, NN), (must, MD), (programmed, VB), (us..."
2,2,"[(game, NN), (must, MD), (feature, VB), (accur..."
3,3,"[(game, NN), (must, MD), (playable, VB), (usin..."
4,4,"[(game, NN), (must, MD), (offer, VB), (compell..."
...,...,...
1957,3224,"[(user, RBR), (able, JJ), (search, NN), (socia..."
1958,3225,"[(user, NN), (see, VBP), (specific, JJ), (soci..."
1959,3226,"[(user, RBR), (able, JJ), (start, JJ), (conver..."
1960,3227,"[(user, RBR), (able, JJ), (send/receive, JJ), ..."


In [182]:
data

,index,Raw Requirements,Requirement with Standard Syntax,type,Domain,sub_domain,raw_length,standard_length,clean_raw_length,clean_standard_length,POS
0,0,game must developed using unity game engine,game shall developed using unity game engine,0,Game,Mobile app,54,51,43,44,"[(game, NN), (must, MD), (developed, VB), (usi..."
1,1,game must programmed using c# language library...,game shall written c# use unity engine-compati...,0,Game,Mobile app,100,74,70,60,"[(game, NN), (must, MD), (programmed, VB), (us..."
2,2,game must feature accurate sound effect music ...,game shall feature accurate sound effect music...,0,Game,Mobile app,92,93,74,75,"[(game, NN), (must, MD), (feature, VB), (accur..."
3,3,game must playable using keyboard key mouse,game shall playable using mouse keyboard.,0,Game,Mobile app,59,52,43,41,"[(game, NN), (must, MD), (playable, VB), (usin..."
4,4,game must offer compelling story convey specia...,game shall offer compelling story convey speci...,0,Game,Mobile app,89,90,62,63,"[(game, NN), (must, MD), (offer, VB), (compell..."
...,...,...,...,...,...,...,...,...,...,...,...
1957,3224,user able search social hub account people typ...,system shall enable user search social hub acc...,0,Social media,Mobile app,96,114,66,81,"[(user, RBR), (able, JJ), (search, NN), (socia..."
1958,3225,user see specific social app? public profile.,system shall enable user see specific social a...,0,Social media,Mobile app,63,88,45,65,"[(user, NN), (see, VBP), (specific, JJ), (soci..."
1959,3226,user able start conversation people searched s...,system shall enable user start conversation pe...,0,Social media,Mobile app,74,92,56,71,"[(user, RBR), (able, JJ), (start, JJ), (conver..."
1960,3227,user able send/receive messages.,system shall enable user send/receive messages.,0,Social media,Mobile app,38,56,32,47,"[(user, RBR), (able, JJ), (send/receive, JJ), ..."


In [183]:

dataa['Nouns'] = 'a'
dataa['Verbs'] = 'b'
dataa['Adjectives']='c'
dataa['Boilerplate']='d'

In [184]:
dataa

,index,POS,Nouns,Verbs,Adjectives,Boilerplate
0,0,"[(game, NN), (must, MD), (developed, VB), (usi...",a,b,c,d
1,1,"[(game, NN), (must, MD), (programmed, VB), (us...",a,b,c,d
2,2,"[(game, NN), (must, MD), (feature, VB), (accur...",a,b,c,d
3,3,"[(game, NN), (must, MD), (playable, VB), (usin...",a,b,c,d
4,4,"[(game, NN), (must, MD), (offer, VB), (compell...",a,b,c,d
...,...,...,...,...,...,...
1957,3224,"[(user, RBR), (able, JJ), (search, NN), (socia...",a,b,c,d
1958,3225,"[(user, NN), (see, VBP), (specific, JJ), (soci...",a,b,c,d
1959,3226,"[(user, RBR), (able, JJ), (start, JJ), (conver...",a,b,c,d
1960,3227,"[(user, RBR), (able, JJ), (send/receive, JJ), ...",a,b,c,d


**Extract** ,**Nouns** **Verbs** **and** **Adjectives***

In [185]:
print(data['POS'][0][0][1])

NN


In [186]:
print(len(data['POS'][0]))

7


In [187]:
for s in range(len(data['POS'][0])):
    if data['POS'][0][s][1]=='NN':
        print(data['POS'][0][s][0])

game
game
engine


In [188]:
for s in range(len(data['POS'][0])):
    if data['POS'][0][s][1]=='VB':
        print(data['POS'][0][s][0])

developed


In [189]:

for s in range(len(data['POS'][0])):
    if data['POS'][0][s][1]=='JJ':
        print(data['POS'][0][s][0])

unity


In [190]:


nouns = []
for s in range(len(data['POS'][0])):
    if data['POS'][0][s][1]=='NN':
        if data['POS'][0][s][0] not in nouns:
            nouns.append(data['POS'][0][s][0])

In [191]:
verbs = []
for s in range(len(data['POS'][0])):
    if data['POS'][0][s][1]=='VB':
        if data['POS'][0][s][0] not in verbs:
            verbs.append(data['POS'][0][s][0])

In [192]:
adjective = []
for s in range(len(data['POS'][0])):
    if data['POS'][0][s][1]=='JJ':
        if data['POS'][0][s][0] not in adjective:
            adjective.append(data['POS'][0][s][0])

In [193]:

nouns

['game', 'engine']

In [194]:
verbs

['developed']

In [195]:
adjective

['unity']

In [196]:
dataa.at[0, 'Nouns'] = nouns

In [197]:

dataa.at[0, 'Verbs'] = verbs

In [198]:
dataa.at[0, 'Adjectives'] = adjective

In [199]:
len(data['POS'])

1962

In [200]:

for t in range(len(data['POS'])):
    nouns = []
    for s in range(len(data['POS'][t])):
        if data['POS'][t][s][1]=='NN':
            if data['POS'][t][s][0] not in nouns:
                nouns.append(data['POS'][t][s][0])
    dataa.at[t, 'Nouns'] = nouns

In [201]:
for t in range(len(data['POS'])):
    verbs = []
    for s in range(len(data['POS'][t])):
        if 'VB' in data['POS'][t][s][1]:
            if data['POS'][t][s][0] not in verbs:
                verbs.append(data['POS'][t][s][0])
    dataa.at[t, 'Verbs'] = verbs

In [202]:
for t in range(len(data['POS'])):
    adjective = []
    for s in range(len(data['POS'][t])):
        if data['POS'][t][s][1]=='JJ':
            if data['POS'][t][s][0] not in adjective:
                adjective.append(data['POS'][t][s][0])
    dataa.at[t, 'Adjectives'] = adjective

In [203]:
len(nouns)

0

In [204]:
len(verbs)

0

In [205]:
len(adjective)

2

In [206]:
print(nouns)


[]


In [207]:
print(verbs)


[]


In [208]:
print(adjective)

['able', 'delete']


In [209]:
dataa

,index,POS,Nouns,Verbs,Adjectives,Boilerplate
0,0,"[(game, NN), (must, MD), (developed, VB), (usi...","[game, engine]","[developed, using]",[unity],d
1,1,"[(game, NN), (must, MD), (programmed, VB), (us...","[game, language, unity, engine]","[programmed, using]","[c, library, compatible]",d
2,2,"[(game, NN), (must, MD), (feature, VB), (accur...","[game, effect, music, conveys]",[feature],"[accurate, sound]",d
3,3,"[(game, NN), (must, MD), (playable, VB), (usin...","[game, key, mouse]","[playable, using]",[keyboard],d
4,4,"[(game, NN), (must, MD), (offer, VB), (compell...","[game, story, convey, message, player]","[offer, compelling]",[special],d
...,...,...,...,...,...,...
1957,3224,"[(user, RBR), (able, JJ), (search, NN), (socia...","[search, hub, account, name, bar]",[typing],"[able, social]",d
1958,3225,"[(user, NN), (see, VBP), (specific, JJ), (soci...","[user, app, profile]",[see],"[specific, social, public]",d
1959,3226,"[(user, RBR), (able, JJ), (start, JJ), (conver...","[conversation, hub]",[searched],"[able, start, social]",d
1960,3227,"[(user, RBR), (able, JJ), (send/receive, JJ), ...",[],[],"[able, send/receive]",d


In [210]:
arr = []
for i in range(len(dataa)):
    verbs = ' '.join(dataa.iloc[i]['Verbs'])
    nouns = ' '.join(dataa.iloc[i]['Adjectives'])
    adjectives = ' '.join(dataa.iloc[i]['Nouns'])
    str = 'The system shall ' + verbs + ' ' + nouns + ' ' + adjectives
    arr.append(str)
arr

['The system shall developed using unity game engine',
 'The system shall programmed using c library compatible game language unity engine',
 'The system shall feature accurate sound game effect music conveys',
 'The system shall playable using keyboard game key mouse',
 'The system shall offer compelling special game story convey message player',
 'The system shall implement offer ai complete game enemy npc experience',
 'The system shall challenge build user complete game think strategy',
 'The system shall  able register system',
 'The system shall  available login website',
 'The system shall  available request cricket tournament user',
 'The system shall  available name tournament system',
 'The system shall starting ending available date tournament',
 'The system shall  contact team player number city name',
 'The system shall assign website user specific id',
 'The system shall  dob email player firstname lastname contact city address',
 'The system shall  specific player id tea

**Boilerplate**

In [211]:

data['Boilerplate Sentence'] = arr
data

,index,Raw Requirements,Requirement with Standard Syntax,type,Domain,sub_domain,raw_length,standard_length,clean_raw_length,clean_standard_length,POS,Boilerplate Sentence
0,0,game must developed using unity game engine,game shall developed using unity game engine,0,Game,Mobile app,54,51,43,44,"[(game, NN), (must, MD), (developed, VB), (usi...",The system shall developed using unity game en...
1,1,game must programmed using c# language library...,game shall written c# use unity engine-compati...,0,Game,Mobile app,100,74,70,60,"[(game, NN), (must, MD), (programmed, VB), (us...",The system shall programmed using c library co...
2,2,game must feature accurate sound effect music ...,game shall feature accurate sound effect music...,0,Game,Mobile app,92,93,74,75,"[(game, NN), (must, MD), (feature, VB), (accur...",The system shall feature accurate sound game e...
3,3,game must playable using keyboard key mouse,game shall playable using mouse keyboard.,0,Game,Mobile app,59,52,43,41,"[(game, NN), (must, MD), (playable, VB), (usin...",The system shall playable using keyboard game ...
4,4,game must offer compelling story convey specia...,game shall offer compelling story convey speci...,0,Game,Mobile app,89,90,62,63,"[(game, NN), (must, MD), (offer, VB), (compell...",The system shall offer compelling special game...
...,...,...,...,...,...,...,...,...,...,...,...,...
1957,3224,user able search social hub account people typ...,system shall enable user search social hub acc...,0,Social media,Mobile app,96,114,66,81,"[(user, RBR), (able, JJ), (search, NN), (socia...",The system shall typing able social search hub...
1958,3225,user see specific social app? public profile.,system shall enable user see specific social a...,0,Social media,Mobile app,63,88,45,65,"[(user, NN), (see, VBP), (specific, JJ), (soci...",The system shall see specific social public us...
1959,3226,user able start conversation people searched s...,system shall enable user start conversation pe...,0,Social media,Mobile app,74,92,56,71,"[(user, RBR), (able, JJ), (start, JJ), (conver...",The system shall searched able start social co...
1960,3227,user able send/receive messages.,system shall enable user send/receive messages.,0,Social media,Mobile app,38,56,32,47,"[(user, RBR), (able, JJ), (send/receive, JJ), ...",The system shall able send/receive


In [212]:
pip install pyspellchecker

Note: you may need to restart the kernel to use updated packages.


In [213]:
from spellchecker import SpellChecker

In [214]:
spell = SpellChecker(distance=1)
def Correct(x):
    return spell.correction(x)
data["Boilerplate Sentence"] = data["Boilerplate Sentence"].apply(Correct)


In [215]:

data.head()

,index,Raw Requirements,Requirement with Standard Syntax,type,Domain,sub_domain,raw_length,standard_length,clean_raw_length,clean_standard_length,POS,Boilerplate Sentence
0,0,game must developed using unity game engine,game shall developed using unity game engine,0,Game,Mobile app,54,51,43,44,"[(game, NN), (must, MD), (developed, VB), (usi...",The system shall developed using unity game en...
1,1,game must programmed using c# language library...,game shall written c# use unity engine-compati...,0,Game,Mobile app,100,74,70,60,"[(game, NN), (must, MD), (programmed, VB), (us...",The system shall programmed using c library co...
2,2,game must feature accurate sound effect music ...,game shall feature accurate sound effect music...,0,Game,Mobile app,92,93,74,75,"[(game, NN), (must, MD), (feature, VB), (accur...",The system shall feature accurate sound game e...
3,3,game must playable using keyboard key mouse,game shall playable using mouse keyboard.,0,Game,Mobile app,59,52,43,41,"[(game, NN), (must, MD), (playable, VB), (usin...",The system shall playable using keyboard game ...
4,4,game must offer compelling story convey specia...,game shall offer compelling story convey speci...,0,Game,Mobile app,89,90,62,63,"[(game, NN), (must, MD), (offer, VB), (compell...",The system shall offer compelling special game...


In [216]:
data.to_csv('file_name.csv', index=False,header=True)


In [217]:
data.columns

Index(['index', 'Raw Requirements', 'Requirement with Standard Syntax',
       ' type', 'Domain', 'sub_domain', 'raw_length', 'standard_length',
       'clean_raw_length', 'clean_standard_length', 'POS',
       'Boilerplate Sentence'],
      dtype='object')

In [218]:
import difflib
text_1='Raw Requirements'
text_2="Requirement with Standard Syntax"
text_3="Boilerplate Sentence"
sequence=difflib.SequenceMatcher(isjunk=None, a=text_1,b=text_2)
difference= sequence.ratio()*100
difference=round(difference, 1)
print(f'{difference} % match')

45.8 % match


In [219]:
Raw = data['Raw Requirements']

In [220]:
Raw[0]

'game must developed using unity game engine'

In [221]:
Standard = data['Requirement with Standard Syntax']

In [222]:
Standard[0]

'game shall developed using unity game engine'

In [223]:
BP = data['Boilerplate Sentence']

In [224]:
BP[0]

'The system shall developed using unity game engine'

In [225]:
data['Raw with Boiler'] = ''
data['Standard with Boiler'] = ''


In [226]:
import difflib

In [227]:
for s in range(data.shape[0]):
    sequence=difflib.SequenceMatcher(isjunk=None, a=Raw[s],b=BP[s])
    try:
        difference= sequence.ratio()*100
    except:
        print("Error in loading raw with boiler {}".format(s))
    difference=round(difference, 1)
    data['Raw with Boiler'].iloc[s]= difference

    sequence=difflib.SequenceMatcher(isjunk=None, a=Standard[s],b=BP[s])
    try:
        difference= sequence.ratio()*100
    except:
        print("error Loading standard with boiler {}".format(s))
    difference=round(difference, 1)
    data['Standard with Boiler'].iloc[s]= difference

Error in loading raw with boiler 203
error Loading standard with boiler 203
Error in loading raw with boiler 207
error Loading standard with boiler 207
Error in loading raw with boiler 272
error Loading standard with boiler 272
Error in loading raw with boiler 279
error Loading standard with boiler 279
Error in loading raw with boiler 352
error Loading standard with boiler 352
Error in loading raw with boiler 353
error Loading standard with boiler 353
Error in loading raw with boiler 354
error Loading standard with boiler 354
Error in loading raw with boiler 356
error Loading standard with boiler 356
Error in loading raw with boiler 366
error Loading standard with boiler 366
Error in loading raw with boiler 467
error Loading standard with boiler 467
Error in loading raw with boiler 469
error Loading standard with boiler 469
Error in loading raw with boiler 473
error Loading standard with boiler 473
Error in loading raw with boiler 481
error Loading standard with boiler 481
Error in loa

In [228]:

data

,index,Raw Requirements,Requirement with Standard Syntax,type,Domain,sub_domain,raw_length,standard_length,clean_raw_length,clean_standard_length,POS,Boilerplate Sentence,Raw with Boiler,Standard with Boiler
0,0,game must developed using unity game engine,game shall developed using unity game engine,0,Game,Mobile app,54,51,43,44,"[(game, NN), (must, MD), (developed, VB), (usi...",The system shall developed using unity game en...,81.7,87.2
1,1,game must programmed using c# language library...,game shall written c# use unity engine-compati...,0,Game,Mobile app,100,74,70,60,"[(game, NN), (must, MD), (programmed, VB), (us...",The system shall programmed using c library co...,72.8,38.3
2,2,game must feature accurate sound effect music ...,game shall feature accurate sound effect music...,0,Game,Mobile app,92,93,74,75,"[(game, NN), (must, MD), (feature, VB), (accur...",The system shall feature accurate sound game e...,69.1,72.9
3,3,game must playable using keyboard key mouse,game shall playable using mouse keyboard.,0,Game,Mobile app,59,52,43,41,"[(game, NN), (must, MD), (playable, VB), (usin...",The system shall playable using keyboard game ...,77.6,64.6
4,4,game must offer compelling story convey specia...,game shall offer compelling story convey speci...,0,Game,Mobile app,89,90,62,63,"[(game, NN), (must, MD), (offer, VB), (compell...",The system shall offer compelling special game...,72.1,75.9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1957,3224,user able search social hub account people typ...,system shall enable user search social hub acc...,0,Social media,Mobile app,96,114,66,81,"[(user, RBR), (able, JJ), (search, NN), (socia...",The system shall typing able social search hub...,54.3,66.7
1958,3225,user see specific social app? public profile.,system shall enable user see specific social a...,0,Social media,Mobile app,63,88,45,65,"[(user, NN), (see, VBP), (specific, JJ), (soci...",The system shall see specific social public us...,70.5,75.2
1959,3226,user able start conversation people searched s...,system shall enable user start conversation pe...,0,Social media,Mobile app,74,92,56,71,"[(user, RBR), (able, JJ), (start, JJ), (conver...",The system shall searched able start social co...,53.4,62.6
1960,3227,user able send/receive messages.,system shall enable user send/receive messages.,0,Social media,Mobile app,38,56,32,47,"[(user, RBR), (able, JJ), (send/receive, JJ), ...",None,62.6,62.6


In [229]:

data.sub_domain.unique()

array(['Mobile app', 'Web development', 'Web & Mobile app',
       'Artificial Intelligence'], dtype=object)

In [230]:
for s in data.sub_domain.unique():
    print(s)

Mobile app
Web development
Web & Mobile app
Artificial Intelligence


In [231]:
sampling = pd.DataFrame(columns=data.sub_domain.unique())

In [232]:
sampling

,Mobile app,Web development,Web & Mobile app,Artificial Intelligence


In [233]:


data.groupby('sub_domain').size()

sub_domain
Artificial Intelligence    141
Mobile app                 592
Web & Mobile app           526
Web development            703
dtype: int64

In [234]:

from random import randint

In [235]:

sampling.columns

Index(['Mobile app', 'Web development', 'Web & Mobile app',
       'Artificial Intelligence'],
      dtype='object')

In [236]:
list=[]
final_samples=[]

In [237]:
import random
for sd in data.sub_domain.unique():
    while True:
        rand = random.randint(0,data.shape[0]-1)
        if data.iloc[rand].sub_domain == sd:
            if rand not in list: 
                list.append(rand)
        if len(list)==1:
            print(list)
            final_samples.extend(list)
            list=[]
            break
        

[839]
[705]
[1022]
[552]


In [238]:
len(data)

1962

In [239]:
val_df = []
for samp in final_samples:
    val = data.iloc[[samp]]
    val_df.append(pd.DataFrame(val))
    #print(val)
    #print(samp)

In [240]:
val_1 = val_df[0].iloc[:,-3:-2]
val_2 = val_df[1].iloc[:,-3:-2]
val_3 = val_df[2].iloc[:,-3:-2]
val_4 = val_df[3].iloc[:,-3:-2]

In [241]:
final_val = pd.concat([val_1, val_2,val_3,val_4])
final_val.head()

,Boilerplate Sentence
839,The system shall allow email send enter system...
705,The system shall conducted paper course code ...
1022,The system shall allow system client project ...
552,None


In [242]:
len(final_samples)

4

In [243]:
NN = []
VB = []
JJ = []

In [244]:
for samp in final_samples:
    for s in range(len(data['POS'][samp])):
        if data['POS'][samp][s][1]=='NN':
            if data['POS'][samp][s][0] not in NN:
                NN.append(data['POS'][samp][s][0])
        if 'VB' in data['POS'][samp][s][1]:
            if data['POS'][samp][s][0] not in VB:
                VB.append(data['POS'][samp][s][0])
        if data['POS'][samp][s][1]=='JJ':
            if data['POS'][samp][s][0] not in JJ:
                JJ.append(data['POS'][samp][s][0])

In [245]:
print(len(NN))
print(NN)

17
['system', 'change', 'password', 'link', 'email', 'paper', 'course', 'code', 'batch', 'day', 'time', 'client', 'project', 'manager', 'view', 'supervisor', 'update']


In [246]:
print(len(VB))
print(VB)

4
['allow', 'email', 'send', 'conducted']


In [247]:
print(len(JJ))
print(JJ)

1
['enter']


In [248]:
new_gen_req = []

In [249]:

for n in range(len(NN)):
    for j in range(len(JJ)):
        for v in range(len(VB)):
            new_gen_req.append('The system shall '+VB[v]+' '+JJ[j]+' '+NN[n])

In [250]:
len(new_gen_req)

68

In [251]:
new_gen_req

['The system shall allow enter system',
 'The system shall email enter system',
 'The system shall send enter system',
 'The system shall conducted enter system',
 'The system shall allow enter change',
 'The system shall email enter change',
 'The system shall send enter change',
 'The system shall conducted enter change',
 'The system shall allow enter password',
 'The system shall email enter password',
 'The system shall send enter password',
 'The system shall conducted enter password',
 'The system shall allow enter link',
 'The system shall email enter link',
 'The system shall send enter link',
 'The system shall conducted enter link',
 'The system shall allow enter email',
 'The system shall email enter email',
 'The system shall send enter email',
 'The system shall conducted enter email',
 'The system shall allow enter paper',
 'The system shall email enter paper',
 'The system shall send enter paper',
 'The system shall conducted enter paper',
 'The system shall allow enter

In [252]:
from sklearn.feature_extraction.text import TfidfVectorizer
X = new_gen_req
tfidf = TfidfVectorizer()
tfidf.fit(X)
transformed = tfidf.transform(X)
X_train = transformed
X_train = X_train.todense()

# Get the name of Features (Feature  Set) and create a DataFrame of Input Features
input_training_features = pd.DataFrame(X_train, columns = tfidf.get_feature_names())
input_training_features.head()

,allow,batch,change,client,code,conducted,course,day,email,enter,...,password,project,send,shall,supervisor,system,the,time,update,view
0,0.663092,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,0.282921,...,0.0,0.0,0.000000,0.282921,0.0,0.565842,0.282921,0.0,0.0,0.0
1,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.637567,0.291182,...,0.0,0.0,0.000000,0.291182,0.0,0.582364,0.291182,0.0,0.0,0.0
2,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,0.282921,...,0.0,0.0,0.663092,0.282921,0.0,0.565842,0.282921,0.0,0.0,0.0
3,0.000000,0.0,0.000000,0.0,0.0,0.663092,0.0,0.0,0.000000,0.282921,...,0.0,0.0,0.000000,0.282921,0.0,0.565842,0.282921,0.0,0.0,0.0
4,0.492667,0.0,0.761927,0.0,0.0,0.000000,0.0,0.0,0.000000,0.210206,...,0.0,0.0,0.000000,0.210206,0.0,0.210206,0.210206,0.0,0.0,0.0


In [253]:
val_2

,Boilerplate Sentence
705,The system shall conducted paper course code ...


In [254]:
values = [val_1,val_2,val_3,val_4]
output_training_features = []
for i in values:
    try:
        y = tfidf.transform(i["Boilerplate Sentence"])
        y_train = y.todense()
        # Get the name of Features (Feature  Set) and create a DataFrame of Input Features
        output_training_features.append(pd.DataFrame(y_train, columns = tfidf.get_feature_names()))
    except:
        print("None")
    

None


In [255]:
len(output_training_features)

3

In [256]:
from sklearn.metrics.pairwise import cosine_similarity
similarities_values = []
for i in range (len(output_training_features)):
    for j in range(len(input_training_features)):
        score = cosine_similarity(output_training_features[i],input_training_features.iloc[j:j+1,:])
        #print("Score for value i ", i," and j ", j, " Score-> ",score)
        similarities_values.append(score)

In [257]:
count = 0
values = []
for i in similarities_values:
    if i < 0.5:
        count = count+1
        values.append(i)
        print(i)
        
print("TOtal Count is ",count)


[[0.40390857]]
[[0.40390857]]
[[0.22631386]]
[[0.43570036]]
[[0.43570036]]
[[0.43570036]]
[[0.44104637]]
[[0.2520558]]
[[0.35597241]]
[[0.2520558]]
[[0.12010558]]
[[0.2520558]]
[[0.35597241]]
[[0.2520558]]
[[0.12010558]]
[[0.2520558]]
[[0.35597241]]
[[0.2520558]]
[[0.12010558]]
[[0.2520558]]
[[0.35597241]]
[[0.2520558]]
[[0.12010558]]
[[0.2520558]]
[[0.35597241]]
[[0.2520558]]
[[0.12010558]]
[[0.2520558]]
[[0.35597241]]
[[0.2520558]]
[[0.12010558]]
[[0.2520558]]
[[0.35597241]]
[[0.2520558]]
[[0.12010558]]
[[0.2520558]]
[[0.35597241]]
[[0.2520558]]
[[0.12010558]]
[[0.2520558]]
[[0.35597241]]
[[0.2520558]]
[[0.12010558]]
[[0.2520558]]
[[0.35597241]]
[[0.2520558]]
[[0.12010558]]
[[0.2520558]]
[[0.35597241]]
[[0.2520558]]
[[0.12010558]]
[[0.2520558]]
[[0.35597241]]
[[0.2520558]]
[[0.12010558]]
[[0.12110501]]
[[0.12464114]]
[[0.12110501]]
[[0.28741527]]
[[0.06748441]]
[[0.06855108]]
[[0.06748441]]
[[0.19105045]]
[[0.06748441]]
[[0.06855108]]
[[0.06748441]]
[[0.19105045]]
[[0.06748441]]
[[0.

In [258]:
print("The Average is ",np.mean(values))

The Average is  0.22516029940605392
